<a href="https://colab.research.google.com/github/zhangxl2002/ORL/blob/main/T5_Ner_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Apr 10 16:34:38 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          Off  | 00000000:00:08.0 Off |                    0 |
|  0%   30C    P8    16W / 150W |      2MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Named Entity Recognition with T5

This notebook shows how to finetune [T5 Model](https://https://huggingface.co/docs/transformers/model_doc/t5) for token classification or named entity recognition with pytorch lighning. In this demo, I used the T5-Small and cast the entities as a text using the text to text framework used in the t5 paper. During Eval the generated tokens are then split and classifies into their specific classes

In [1]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
# import pytorch_lightning as pl


from transformers import (
    AdamW,
    MT5ForConditionalGeneration,
    T5ForConditionalGeneration,
    T5Tokenizer,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

from datasets import load_dataset

/home/pai/envs/T5/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [3]:
dataset = load_dataset("zhangxl2002/mpqa_ORL")

In [5]:
print(dataset)
cnt = 0
for i in range(len(dataset['train'])):
    if len(dataset['train'][i]['spans']) > 3:
        print(dataset['train'][i])
        cnt+=1
        if cnt>5: break

print(" ".join(dataset['train'][0]['words']))

DatasetDict({
    train: Dataset({
        features: ['words', 'label_ids', 'labels', 'spans', 'dse'],
        num_rows: 3549
    })
    validation: Dataset({
        features: ['words', 'label_ids', 'labels', 'spans', 'dse'],
        num_rows: 893
    })
    test: Dataset({
        features: ['words', 'label_ids', 'labels', 'spans', 'dse'],
        num_rows: 1509
    })
})
{'words': ['He', 'argued', 'that', 'had', 'the', 'West', 'not', 'continued', 'to', 'keep', 'alive', 'during', 'the', 'past', 'several', 'years', 'the', 'Cold', 'War', 'stereotype', 'of', 'a', 'threat', 'from', 'the', 'East', ',', 'but', 'would', 'have', 'concentrated', 'instead', 'on', 'terrorism', ',', 'the', 'common', 'enemy', ',', 'the', 'twin', 'towers', 'of', 'New', 'York', 'may', 'not', 'have', 'collapsed', '.'], 'label_ids': [1, 3, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0], 'labels': ['B-AGENT', 'B-DSE', 'O',

In [4]:
class MPQADataset(Dataset):
  def __init__(self, tokenizer, dataset, type_path, max_len=512):

    self.data = dataset[type_path]
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.tokenizer.max_length = max_len
    self.tokenizer.model_max_length = max_len
    self.inputs = []
    self.targets = []

    self._build()

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

  def _build(self):
    for idx in range(len(self.data)):
      input_, target = " ".join(self.data[idx]["words"]), "; ".join(self.data[idx]["spans"])
      input_ = input_ + " DSE:" + self.data[idx]["dse"]

      input_ = input_.lower() + ' </s>'
      target = target.lower() + " </s>"

       # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [input_], max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
      )
       # tokenize targets
      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target],max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("../T5-base")

print(tokenizer)

input_dataset = MPQADataset(tokenizer=tokenizer, dataset=dataset, type_path='train')

T5TokenizerFast(name_or_path='../T5-base', vocab_size=32100, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42

In [8]:
data = input_dataset[0]
# print(data)
print(tokenizer.decode(data["source_ids"][400]))
print((tokenizer.decode(data["source_ids"][0:5])))
print(tokenizer.encode("<unk>"))
print(data["source_ids"][0:2])
print(tokenizer.pad_token_id)
print(tokenizer.decode(data["source_ids"], skip_special_tokens=False))
print(tokenizer.decode(data["target_ids"], skip_special_tokens=False))

<pad>
the kimberley
[2, 1]
tensor([8, 3])
0
the kimberley provincial hospital said it would probably know by tuesday whether one of its patients had congo fever. dse:would probably know</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [6]:
args_dict = dict(
    data_dir="zhangxl2002/mpqa_ORL", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=10,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=True, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

In [7]:
args = argparse.Namespace(**args_dict)

In [8]:
class T5FineTuner():
    def __init__(self, hparam):
        self.hparam = hparam
        # self.model = T5ForConditionalGeneration.from_pretrained(
        #     hparam.model_name_or_path)
        self.model = T5ForConditionalGeneration.from_pretrained(
            "saved_models/model_epoch_7")
        self.tokenizer = AutoTokenizer.from_pretrained(
            hparam.model_name_or_path
        )
        self.configure_optimizers()
    def is_logger(self):
        return True

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self.model(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        # tensorboard_logs = {"train_loss": loss}
        # return {"loss": loss, "log": tensorboard_logs}
        return loss

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            loss = self._step(batch)
        return loss

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparam.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparam.learning_rate, eps=self.hparam.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self,
                       epoch=None,
                       batch_idx=None,
                       optimizer=None,
                       optimizer_idx=None,
                       optimizer_closure=None,
                       on_tpu=None,
                       using_native_amp=None,
                       using_lbfgs=None
                       ):

        # optimizer.step(closure=optimizer_closure)
        # optimizer.zero_grad()
        self.opt.step(closure=optimizer_closure)
        self.opt.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(
            self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(
            tokenizer=self.tokenizer, type_path="train", args=self.hparam)
        dataloader = DataLoader(train_dataset, batch_size=self.hparam.train_batch_size,
                                drop_last=True, shuffle=True, num_workers=2)
        t_total = (
            (len(dataloader.dataset) //
             (self.hparam.train_batch_size * max(1, self.hparam.n_gpu)))
            // self.hparam.gradient_accumulation_steps
            * float(self.hparam.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparam.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(
            tokenizer=self.tokenizer, type_path="validation", args=self.hparam)
        return DataLoader(val_dataset, batch_size=self.hparam.eval_batch_size, num_workers=2)

In [22]:
model = T5FineTuner(args)

In [36]:
# from transformers import T5ForConditionalGeneration, T5Config
# config = T5Config.from_pretrained("t5-base")
# # 创建一个与原始模型相同架构的模型实例
# model = T5ForConditionalGeneration(config=config)

# # 加载之前保存的PyTorch模型的参数
# model.load_state_dict(torch.load("saved_models/model_epoch_3.pth"))

# # 将模型保存为Hugging Face的格式
# model.save_pretrained("./test_saved_models")

# # 模型现在已经转换为Hugging Face的形式并保存到指定路径下

In [15]:
checkpointPath = "saveCheckpointPath/checkpoint.pth"

In [9]:
def get_dataset(tokenizer, type_path, args):
    tokenizer.max_length = args.max_seq_length
    tokenizer.model_max_length = args.max_seq_length
    # dataset = load_dataset(args.data_dir)
    dataset = load_dataset("zhangxl2002/mpqa_ORL")
    return MPQADataset(tokenizer=tokenizer, dataset=dataset, type_path=type_path)

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.model.to(device)
# for i, batch in enumerate(dataloader):
#     # x, y = batch                      moved to training_step
#     # y_hat = model(x)                  moved to training_step
#     # loss = loss_function(y_hat, y)    moved to training_step
#     loss = lightning_module.training_step(batch, i)

#     # Lighting handles automatically:
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
max_epochs = 10
train_dataloader = model.train_dataloader()
val_dataloader = model.val_dataloader()
for epoch in range(max_epochs):
    # 在验证集上计算损失
    val_loss = 0.0
    num_val_batches = 0
    for i, val_batch in enumerate(val_dataloader):
        val_batch = {key: value.to(device) for key, value in val_batch.items()}  # 将批次数据移动到GPU上
        val_loss += model.validation_step(val_batch, i).item()
        num_val_batches += 1
    avg_val_loss = val_loss / num_val_batches
    print("---------------------------------------")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print("---------------------------------------")
    
    total_loss = 0.0
    num_batches = 0
    for i, batch in enumerate(train_dataloader):
        batch = {key: value.to(device) for key, value in batch.items()}  # 将批次数据移动到GPU上
        loss = model.training_step(batch, i)
        loss.backward()
        total_loss += loss.item()
        num_batches += 1

        # 打印每个批次的损失值
        print(f"Epoch [{epoch + 1}/{max_epochs}], Batch [{i + 1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

        # 执行优化步骤
        model.optimizer_step()
    # 计算并打印平均损失值
    avg_loss = total_loss / num_batches
    print(f"Epoch [{epoch + 1}/{max_epochs}], Average Loss: {avg_loss:.4f}")

    # 在验证集上计算损失
    model.model.eval()
    val_loss = 0.0
    num_val_batches = 0
    for i, val_batch in enumerate(val_dataloader):
        val_batch = {key: value.to(device) for key, value in val_batch.items()}  # 将批次数据移动到GPU上
        val_loss += model.validation_step(val_batch, i).item()
        num_val_batches += 1
    avg_val_loss = val_loss / num_val_batches
    print("---------------------------------------")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print("---------------------------------------")
    model.model.train()


    save_dir = f"./saved_models/model_epoch_{epoch + 1}_trainloss_{avg_loss}_valloss_{avg_val_loss}"
    model.model.save_pretrained(save_dir)
    print(f"Model saved at {save_dir}")

    

    

In [ ]:
# 使用AL对训练过程进行改写

In [10]:
from ALstrategy import *

In [69]:

from torch.utils.data import Subset
from torch.utils.data import ConcatDataset

class Datapool():
    def __init__(self, warmstart_percentage, train_dl):
        # self.train_dl = train_dl #
        print("total_num:",len(train_dl.dataset))
        self.train_data = self.dataloader_to_list(train_dl)
        self.total_num = len(self.train_data)
        print("total_num:",self.total_num)
        annotated_num = int(self.total_num * warmstart_percentage)
        annotated_indices = random.sample(range(self.total_num), annotated_num)
        annotated_indices.sort()
        self.annotated_data = [self.train_data[i] for i in annotated_indices]
        self.unannotated_data = [self.train_data[i] for i in range(self.total_num) if i not in annotated_indices]
        print("len(annotated_data)",len(self.annotated_data))
        print("len(unannotated_data)",len(self.unannotated_data))
        print("self.annotated_data[0]",self.annotated_data[0])

        self.an_dataloader = DataLoader(self.annotated_data, batch_size=8, shuffle=False, drop_last=True, num_workers=2)
        self.un_dataloader = DataLoader(self.unannotated_data, batch_size=8, shuffle=False, drop_last=True, num_workers=2)

        # self.addAnnotatedData(0.02)


    def dataloader_to_list(self, dataloader):
        data_list = []
        cnt = 0
        for batch in dataloader:  
            for i in range(dataloader.batch_size):      
                data_list.append({
                    'source_ids':batch['source_ids'][i],
                    'target_ids':batch['target_ids'][i],
                    'source_mask':batch['source_mask'][i],
                    'target_mask':batch['target_mask'][i]
                })
        return data_list

    def getAnnotatedData(self):
        return self.annotated_data
    def getAnnotatedDataloader(self):
        return self.an_dataloader
    def getUnannotatedData(self):
        return self.unannotated_data
    def getUnannotatedDataloader(self):
        return self.un_dataloader
    def addAnnotatedData(self, add_percentage=0.0, selected_indices=[]):
        # 采用随机策略或者直接使用外部传入的下标
        if len(selected_indices) == 0:
            add_num = int(self.total_num * add_percentage)
            annotated_indices = random.sample(range(len(self.unannotated_data)), add_num)
        else:
            annotated_indices = selected_indices
        annotated_indices.sort()
        self.annotated_data = self.annotated_data + [self.unannotated_data[i] for i in annotated_indices]
        self.unannotated_data = [self.unannotated_data[i] for i in range(len(self.unannotated_data)) if i not in annotated_indices]
        self.an_dataloader = DataLoader(self.annotated_data, batch_size=8, shuffle=False, drop_last=True, num_workers=2)
        self.un_dataloader = DataLoader(self.unannotated_data, batch_size=8, shuffle=False, drop_last=True, num_workers=2)
        print("After adding, annotated_samples_num:", len(self.an_dataloader.dataset))
        print("After adding, unannotated_samples_num:", len(self.un_dataloader.dataset))


In [73]:
class T5FineTunerWithAL(T5FineTuner):
    def __init__(self, hparam, strategy_name="RANDOM", warmstart_percentage=0.01):
        super().__init__(hparam)
        self.strategy = {
            "RANDOM": RandomStrategy,
            # "LC": LeastConfidenceStrategy,
            # "NLC": NormalizedLeastConfidenceStrategy,
            # "LTP": LeastTokenProbabilityStrategy,
            # "MTP": MinimumTokenProbabilityStrategy,
            # "MTE": MaximumTokenEntropyStrategy,
            # "LONG": LongStrategy,
            # "TE": TokenEntropyStrategy,
        }
        self.strategy_name = strategy_name
        self.strategy = self.strategy[strategy_name]
        self.datapool = Datapool(warmstart_percentage, super().train_dataloader())
        print(f"T5FineTunerWithAL init: use strategy{strategy_name}")
    def resetModel():
        # 重置模型权重
        del self.model
        self.model = T5ForConditionalGeneration.from_pretrained(
            "saved_models/model_epoch_7")
        # 重置优化器
        del self.opt
        self.configure_optimizers()
    # def forward(
    #     self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
    # ):
    #     return self.model(
    #         input_ids,
    #         attention_mask=attention_mask,
    #         decoder_input_ids=decoder_input_ids,
    #         decoder_attention_mask=decoder_attention_mask,
    #         labels=lm_labels,
    #     )

    # def _step(self, batch):
    #     lm_labels = batch["target_ids"]
    #     lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        # outputs = self.model(
        #     input_ids=batch["source_ids"],
        #     attention_mask=batch["source_mask"],
        #     labels=lm_labels,
        #     decoder_attention_mask=batch['target_mask']
        # )

    #     loss = outputs[0]

    #     return loss

    # def training_step(self, batch, batch_idx):
    #     loss = self._step(batch)
    #     return loss

    # def validation_step(self, batch, batch_idx):
    #     with torch.no_grad():
    #         loss = self._step(batch)
    #     return loss

    # def train_dataloader(self):
    #     train_dataset = get_dataset(
    #         tokenizer=self.tokenizer, type_path="train", args=self.hparam)
    #     dataloader = DataLoader(train_dataset, batch_size=self.hparam.train_batch_size,
    #                             drop_last=True, shuffle=True, num_workers=2)
    #     t_total = (
    #         (len(dataloader.dataset) //
    #          (self.hparam.train_batch_size * max(1, self.hparam.n_gpu)))
    #         // self.hparam.gradient_accumulation_steps
    #         * float(self.hparam.num_train_epochs)
    #     )
    #     scheduler = get_linear_schedule_with_warmup(
    #         self.opt, num_warmup_steps=self.hparam.warmup_steps, num_training_steps=t_total
    #     )
    #     self.lr_scheduler = scheduler
    #     return dataloader
    def update_datapool(self, strategy="RANDOM", add_percentage=0.0):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        un_data = self.datapool.getUnannotatedData()
        self.model.to(device)
        cnt = 0

        un_dataloader = DataLoader(un_data, batch_size=8, shuffle=False)
        
        # for i in range(len(un_data)):
        #     outputs = model.model.generate(input_ids=un_data[i]["source_ids"],\
        #                 attention_mask=un_data[i]["source_mask"],return_dict_in_generate=True,output_scores=True,max_length=100)
        #     #output.scores是一个长度为输出的token数量的元组，元组中的每个元素为[batchsize,词表大小]的tensor
        #     transition_scores = model.model.compute_transition_scores(
        #         outputs.sequences, outputs.scores, normalize_logits=True
        #     )

        avg_probs=[]
        min_probs=[]
        idx = 0
        for i, batch in enumerate(un_dataloader):
                batch = {key: value.to(device) for key, value in batch.items()} 
                outputs = model.model.generate(input_ids=batch["source_ids"],\
                        attention_mask=batch["source_mask"],return_dict_in_generate=True,output_scores=True,max_length=100)
                
                #output.scores是一个长度为输出的token数量的元组，元组中的每个元素为[batchsize,词表大小]的tensor
                transition_scores = model.model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                )
                # compute_transition_scores计算出来的transition_scores是一个[batchsize,输出的token数量]的tensor，相当于对于每个输出的token一个评分

                # input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
                # 由于t5是encoder-decoder架构，所以这里input_length取1
                generated_tokens = outputs.sequences[:, 1:] 
                # generated_tokens是一个[batchsize,输出的token数量]的tensor，相当于每一个输出的token对应的id

                # 遍历batch中的每一条样本，计算对每一条样本的不确定程度
                for i in range(generated_tokens.shape[0]):
                    total_probs = 0.0 # 可能性的总和
                    valid_num = 0 # 实际的token的数量
                    min_prob = 1.0
                    for tok, score in zip(generated_tokens[i], transition_scores[i]):
                        if tok == 0:
                            break
                        # | token | token string | logits | probability
                        # print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.4f} |{np.exp(score.cpu().numpy()):.2%}")
                        total_probs += np.exp(score.cpu().numpy())
                        min_prob = min(min_prob, np.exp(score.cpu().numpy()))
                        valid_num += 1
                    if valid_num > 0:
                        # print("average prob:", total_probs/valid_num)
                        # print("min prob:", min_prob)
                        avg_probs.append((idx, total_probs/valid_num))
                        min_probs.append((idx, min_prob))
                        idx += 1
                    
                    # print(tokenizer.decode(outputs.sequences[i], skip_special_tokens=True))
                    # print("----------------------------------------------")
        # 得到不确定指标(如avg_probs,min_probs后,进行排序)
        sorted_avg_probs = sorted(avg_probs, key=lambda x: x[1])
        sorted_min_probs = sorted(min_probs, key=lambda x: x[1])
        print("sorted_avg_probs[:10]:",sorted_avg_probs[:10])
        print("sorted_min_probs[:10]:",sorted_min_probs[:10])
        if add_percentage > 0:
            if strategy == "RANDOM":
                self.datapool.addAnnotatedData(add_percentage)
            elif strategy == "AVG":
                sorted_avg_probs = sorted(avg_probs, key=lambda x: x[1])
                selected_idx = [item[0] for item in sorted_avg_probs][:int(len(sorted_avg_probs) * add_percentage)]
                self.datapool.addAnnotatedData(add_percentage, selected_idx)
            elif strategy == "MIN":
                sorted_min_probs = sorted(min_probs, key=lambda x: x[1])
                selected_idx = [item[0] for item in sorted_min_probs][:int(len(sorted_min_probs) * add_percentage)]
                self.datapool.addAnnotatedData(add_percentage, selected_idx)


        # print(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))
        

    # def val_dataloader(self):
    #     val_dataset = get_dataset(
    #         tokenizer=self.tokenizer, type_path="validation", args=self.hparam)
    #     return DataLoader(val_dataset, batch_size=self.hparam.eval_batch_size, num_workers=2)


In [74]:
model = T5FineTunerWithAL(args)

/home/pai/envs/T5/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


total_num: 3549


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total_num: 3544
len(annotated_data) 35
len(unannotated_data) 3509
self.annotated_data[0] {'source_ids': tensor([19236,     3,     6,  2662,  2837,     3,     6,  2839,   165,  1518,
           48,    97,     3,     6,  8560,     8,  6488,  4750,    12,   474,
            8,     3,     2,     3,  3781,    32,   235, 10015,     3,    31,
           31,   139,  2054,    11,   237,   718,    21,  4400,   257,    13,
            8,  6488,     3,     5,     3,    26,     7,    15,    10, 28236,
         3843,     1,     1,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,   

In [75]:
model.update_datapool(strategy="AVG",add_percentage=0.02)

sorted_avg_probs[:10]: [(3226, 0.34628525376319885), (2311, 0.3589614927768707), (2985, 0.3770226538181305), (1612, 0.39566075801849365), (3220, 0.3960336744785309), (2218, 0.39842841029167175), (1996, 0.4008737802505493), (2360, 0.4045694172382355), (1663, 0.4169021248817444), (2851, 0.4214211106300354)]
sorted_min_probs[:10]: [(2827, 0.12535971), (881, 0.14681637), (2288, 0.19831336), (2747, 0.212865), (2238, 0.21288015), (2760, 0.213783), (367, 0.2183564), (3107, 0.2183564), (2448, 0.22252536), (593, 0.23810562)]
After adding, annotated_samples_num: 105
After adding, unannotated_samples_num: 3439


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


max_epochs = 10 
max_iters = 10 # 每个iterator选择新的数据加入已标注数据的集合并重新训练模型

# 初始化，默认初始标注数据占总数量的1%，使用RANDOM选择策略
model = T5FineTunerWithAL(args) 

# 验证集始终是一样的
val_dataloader = model.val_dataloader()

for iter in range(max_iters):
    # 对每轮新的数据都重新训练
    model.resetModel()
    model.model.to(device)

    # 训练集每轮更新，需要重新加载
    annotated_dataloader = model.annotated_dataloader()

    for epoch in range(max_epochs):
        # 训练
        total_loss = 0.0
        num_batches = 0
        for i, batch in enumerate(annotated_dataloader):
            batch = {key: value.to(device) for key, value in batch.items()}  # 将批次数据移动到GPU上
            loss = model.training_step(batch, i)
            loss.backward()
            total_loss += loss.item()
            num_batches += 1

            # 打印每个批次的损失值
            print(f"Epoch [{epoch + 1}/{max_epochs}], Batch [{i + 1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

            # 执行优化步骤
            model.optimizer_step()

        # 计算并打印平均损失值
        avg_loss = total_loss / num_batches
        print(f"Epoch [{epoch + 1}/{max_epochs}], Average Loss: {avg_loss:.4f}")


        # 每一个epoch训练完成后，在验证集上计算损失
        val_loss = 0.0
        num_val_batches = 0
        for i, val_batch in enumerate(val_dataloader):
            val_batch = {key: value.to(device) for key, value in val_batch.items()} 
            val_loss += model.validation_step(val_batch, i).item()
            num_val_batches += 1
        avg_val_loss = val_loss / num_val_batches
        print(f"----------Validation Loss: {avg_val_loss:.4f}-------------")

        # TODO: 保存当前迭代当前epoch的权重

    # TODO:所有epoch结束后，对未标注数据采用选择策略，选出新的标注数据，更新datapool
    model.update_datapool(strategy="AVG",0.02)



train_dataloader = model.train_dataloader()
val_dataloader = model.val_dataloader()
for epoch in range(max_epochs):
    # 在验证集上计算损失
    val_loss = 0.0
    num_val_batches = 0
    for i, val_batch in enumerate(val_dataloader):
        val_batch = {key: value.to(device) for key, value in val_batch.items()}  # 将批次数据移动到GPU上
        val_loss += model.validation_step(val_batch, i).item()
        num_val_batches += 1
    avg_val_loss = val_loss / num_val_batches
    print("---------------------------------------")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print("---------------------------------------")
    
    total_loss = 0.0
    num_batches = 0
    for i, batch in enumerate(train_dataloader):
        batch = {key: value.to(device) for key, value in batch.items()}  # 将批次数据移动到GPU上
        loss = model.training_step(batch, i)
        loss.backward()
        total_loss += loss.item()
        num_batches += 1

        # 打印每个批次的损失值
        print(f"Epoch [{epoch + 1}/{max_epochs}], Batch [{i + 1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

        # 执行优化步骤
        model.optimizer_step()
    # 计算并打印平均损失值
    avg_loss = total_loss / num_batches
    print(f"Epoch [{epoch + 1}/{max_epochs}], Average Loss: {avg_loss:.4f}")

    # 在验证集上计算损失
    model.model.eval()
    val_loss = 0.0
    num_val_batches = 0
    for i, val_batch in enumerate(val_dataloader):
        val_batch = {key: value.to(device) for key, value in val_batch.items()}  # 将批次数据移动到GPU上
        val_loss += model.validation_step(val_batch, i).item()
        num_val_batches += 1
    avg_val_loss = val_loss / num_val_batches
    print("---------------------------------------")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print("---------------------------------------")
    model.model.train()


    save_dir = f"./saved_models/model_epoch_{epoch + 1}_trainloss_{avg_loss}_valloss_{avg_val_loss}"
    model.model.save_pretrained(save_dir)
    print(f"Model saved at {save_dir}")


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
val_dataloader = model.val_dataloader()
model.model.to(device)
cnt = 0
for i, batch in enumerate(val_dataloader):
        batch = {key: value.to(device) for key, value in batch.items()} 
        outputs = model.model.generate(input_ids=batch["source_ids"],\
                attention_mask=batch["source_mask"],return_dict_in_generate=True,output_scores=True,max_length=100)
        
        cnt += 1
        if cnt > 1:
            break
        # if len(outputs.scores) == 19 :
        #     continue
        

        print("outputs.keys(): ",outputs.keys())
        print("len(outputs.scores)",len(outputs.scores))
        print("outputs.scores[0].shape:",outputs.scores[0].shape)
        print("len(outputs.scores):",len(outputs.scores))
        #output.scores是一个长度为输出的token数量的元组，元组中的每个元素为[batchsize,词表大小]的tensor
        print("outputs.scores[0].shape:",outputs.scores[0].shape) 
        print("ddddddddddddddddddddddddddddddddddddd",outputs.scores[0][0][0])

        transition_scores = model.model.compute_transition_scores(
            outputs.sequences, outputs.scores, normalize_logits=True
        )
        print("transition_scores.shape:", transition_scores.shape)
        # compute_transition_scores计算出来的transition_scores是一个[batchsize,输出的token数量]的tensor，相当于对于每个输出的token一个评分
        print("len(transition_scores[0]): ", len(transition_scores[0]))
        # print("transition_scores: ", transition_scores)

        # input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
        # 由于t5是encoder-decoder架构，所以这里input_length取1
        generated_tokens = outputs.sequences[:, 1:] 
        # generated_tokens是一个[batchsize,输出的token数量]的tensor，相当于每一个输出的token对应的id
        print("outputs.sequences: ", outputs.sequences)
        print("len(generated_tokens[0]):",len(generated_tokens[0]))
        print("generated_tokens: ",generated_tokens)

        # 遍历batch中的每一条样本，计算对每一条样本的不确定程度
        for i in range(generated_tokens.shape[0]):
            total_probs = 0.0 # 可能性的总和
            valid_num = 0 # 实际的token的数量
            min_probs = 1.0
            for tok, score in zip(generated_tokens[i], transition_scores[i]):
                if tok == 0:
                    break
                # | token | token string | logits | probability
                print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.4f} |{np.exp(score.cpu().numpy()):.2%}")
                total_probs += np.exp(score.cpu().numpy())
                min_probs = min(min_probs, np.exp(score.cpu().numpy()))
                valid_num += 1
            if valid_num > 0:
                print("average prob:", total_probs/valid_num)
                print("min prob:", min_probs)
            
            print(tokenizer.decode(outputs.sequences[i], skip_special_tokens=True))
            print("----------------------------------------------")
            #TODO 计算出不确定程度度量
        
        # print(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


outputs.keys():  odict_keys(['sequences', 'scores', 'past_key_values'])
len(outputs.scores) 35
outputs.scores[0].shape: torch.Size([8, 32128])
len(outputs.scores): 35
outputs.scores[0].shape: torch.Size([8, 32128])
ddddddddddddddddddddddddddddddddddddd tensor(-26.1285, device='cuda:0')
transition_scores.shape: torch.Size([8, 35])
len(transition_scores[0]):  35
outputs.sequences:  tensor([[    0,  2387,    10,  1045,  2371,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [    0,  2387,    10,     3,     7,   232,    45,  8390,   844,     3,
             6,   902,     8,  4723,  1911,  7579,    23,     9, 21286,  1719,
             3,     6,    11, 12848,  5784,    45,     8,   690,     3,    31,
             7,     3, 24530,  1449,  1471,     1],
        [    0,  3102,    10,     3, 26

In [80]:
print("len(outputs.scores)",len(outputs.scores))
print(batch.keys())
print(batch["source_ids"].shape)
print(batch["source_mask"].shape)
print(batch["target_ids"].shape)

len(outputs.scores) 19
dict_keys(['source_ids', 'source_mask', 'target_ids', 'target_mask'])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])


In [16]:
logger = logging.getLogger(__name__)

class LoggingCallback():
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [17]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filename=args.output_dir+"/checkpoint.pth", monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    #early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    #amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

# train_params = dict(
#     accumulate_grad_batches=args.gradient_accumulation_steps,
#     ## gpus=args.n_gpu,
#     max_epochs=args.num_train_epochs,
#     #early_stop_callback=False,
#     precision= 16 if args.fp_16 else 32,
#     #amp_level=args.opt_level,
#     gradient_clip_val=args.max_grad_norm,
#     # callbacks=[LoggingCallback()],
# )

In [22]:
trainer = pl.Trainer(**train_params)

Using 16bit native Automatic Mixed Precision (AMP)
/home/pai/envs/T5/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fb6a2a8c9d0>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fb6a2a8c9d0>)`.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model)

In [25]:
# model = model.load_from_checkpoint("/mnt/workspace/ORL/lightning_logs/version_4/checkpoints/epoch=9-step=279.ckpt")
model = model.model.from_pretrained("/mnt/workspace/ORL/saved_models/model_epoch_2_trainloss_0.05876155760408323_valloss_0.23555645180333937")

In [16]:
import textwrap

dataloader = DataLoader(input_dataset, batch_size=32, num_workers=2, shuffle=True)
model.eval()
model = model.to("cpu")

In [17]:
outputs = []
targets = []
texts = []
cnt = 0
for batch in dataloader:

    outs = model.generate(input_ids=batch['source_ids'],
                                attention_mask=batch['source_mask'])
    dec = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip() for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
                for ids in batch["target_ids"]]
    text = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
                for ids in batch["source_ids"]]
    texts.extend(text)
    outputs.extend(dec)
    targets.extend(target)
    break


for i in range(10):
    c = texts[i]
    lines = textwrap.wrap("text:\n%s\n" % c, width=100)
    print("\n".join(lines))
    print("\nActual Entities: %s" % target[i])
    print("Predicted Entities: %s" % outputs[i])
    print("=====================================================================\n")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/pai/envs/T5/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


text: i do n't , just so i can work in the school i 'll be happy . dse:happy

Actual Entities: target: work in the school; agent: i
Predicted Entities: agent: i

text: the uk foreign secretary , jack straw , said :  for months the government of zimbabwe has
conducted a systematic campaign of violence and intimidation , designed to achieve an outcome -
power at all costs . '' dse:said

Actual Entities: agent: the uk foreign secretary , jack straw; target: the government of zimbabwe
Predicted Entities: agent: the uk foreign secretary , jack straw; target: the government of zi

text: in the citizen , columnist rosa harris-adler wrote a devastatingly sarcastic piece ,
ridiculing mr. tobin 's assertion that he was leaving for family reasons . dse:ridiculing

Actual Entities: agent: rosa harris-adler
Predicted Entities: agent: rosa harris-adler; target: mr.

text: they 're actually going to paint arrows on the floors of the cells so they 'll know to face
north , '' he says . dse:says

Actual

In [ ]:
dir(model.model)

In [118]:
# search:how to get the probability huggingface
# https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075

# 设置输入句子和相关参数
input_text = "I love the person who likes me. dse:love"
input_texts = ["I love the person who likes me. dse:love","He loves his wife. dse:loves"]
max_length = 32
temperature = 1.0
num_samples = 1

# 推理
# inputs = tokenizer([input_text], return_tensors="pt")

inputs = tokenizer.batch_encode_plus(
    input_texts, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt"
)
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.model.generate(**inputs,return_dict_in_generate=True,output_scores=True)
# print(outputs)
print("outputs.keys(): ",outputs.keys())
print("outputs.scores[0].shape:",outputs.scores[0].shape)
print("len(outputs.scores):",len(outputs.scores))
#output.scores是一个长度为输出的token数量的元组，元组中的每个元素为batchsize*词表大小的tensor
print("outputs.scores[0].shape:",outputs.scores[0].shape) 
print("ddddddddddddddddddddddddddddddddddddd",outputs.scores[0][0][0])

transition_scores = model.model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
print("len(transition_scores[0]): ", len(transition_scores[0]))
print("transition_scores: ", transition_scores)

# input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
# 由于t5是encoder-decoder架构，所以这里input_length取1
generated_tokens = outputs.sequences[:, 1:] 
print("outputs.sequences: ", outputs.sequences)
print("len(generated_tokens[0]):",len(generated_tokens[0]))
print("generated_tokens: ",generated_tokens)
for i in range(generated_tokens.shape[0]):
    for tok, score in zip(generated_tokens[i], transition_scores[i]):
        # | token | token string | logits | probability
        print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.4f} |{np.exp(score.numpy()):.2%}")
    print("----------------------------------------------")

print(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))



/home/pai/envs/T5/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


outputs.keys():  odict_keys(['sequences', 'scores', 'past_key_values'])
outputs.scores[0].shape: torch.Size([2, 32128])
len(outputs.scores): 13
outputs.scores: torch.Size([2, 32128])
ddddddddddddddddddddddddddddddddddddd tensor(-22.2044)
len(transition_scores[0]):  13
transition_scores:  tensor([[-7.3570e-02, -1.5182e-04, -1.8435e-02, -1.8597e-01, -1.7682e-03,
         -9.1794e-05, -1.1499e-02, -1.4008e-04, -2.0066e-01, -2.2774e-02,
         -1.1470e-03, -4.0414e-04, -5.3514e-03],
        [-9.9976e-02, -7.1911e-05, -1.6243e-02, -1.4897e-01, -1.2826e-04,
         -1.0216e-04, -2.3009e-03, -3.8949e-05, -8.3902e-03, -5.1038e+01,
         -2.2564e+01, -2.2113e+01, -2.1953e+01]])
outputs.sequences:  tensor([[   0, 3102,   10,   27,  117, 2387,   10,    8,  568,  113,  114,    7,
          140,    1],
        [   0, 3102,   10,  216,  117, 2387,   10,  112, 2512,    1,    0,    0,
            0,    0]])
len(generated_tokens[0]): 13
generated_tokens:  tensor([[3102,   10,   27,  117, 2387,   

In [33]:
for batch in dataloader:

    outs = model.model.generate(input_ids=batch['source_ids'],
                                attention_mask=batch['source_mask'])
    # dec = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip() for ids in outs]
    # target = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
    #             for ids in batch["target_ids"]]
    # text = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
    #             for ids in batch["source_ids"]]
    # texts.extend(text)
    # outputs.extend(dec)
    # targets.extend(target)
    # cnt += 1
    # if cnt > 10:
    #     break
    print(outs)
    print(tokenizer.decode(outs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip())
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/pai/envs/T5/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    0,  2387,    10,     8, 23997,  3368,   628,  2478,     1,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0,  3102,    10,     3,    23,   117,  2387,    10,     3,  1258,
           189,    63,   410,    48, 24522,     1,     0,     0,     0,     0],
        [    0,  3102,    10,     8,   126, 25453,   648,     3,     6, 10381,
         11831,    15,     7,   648,    11,  6179,  6029,   442,   117,  2387],
        [    0,  2387,    10,     8,     3,   102,    29,   102,     1,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0,  3102,    10,  7609,   261,    12,     8,  1075,    13,  1291,
          4028,   127,     7,   117,  2387,    10,  4297,  3101,    84,  4840],
        [    0,  3102,    10,   112,   117,  2387,    10, 26504,   323,    30,
             8,   962,     1,     0,     0,     0,     0,     0,     0,     0],
        [    0,  3102,    10,     3,     9,  1

In [41]:
print(outs[0])
print(tokenizer.decode(outs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip())

print(outs[1])
print(tokenizer.decode(outs[1], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip())
for i in range(15):
    print(outs[1][i])
    print(tokenizer.decode(outs[1][i], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip())

tensor([    0,  2387,    10,     8, 23997,  3368,   628,  2478,     1,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
target: the christmas island space station
tensor([    0,  3102,    10,     3,    23,   117,  2387,    10,     3,  1258,
          189,    63,   410,    48, 24522,     1,     0,     0,     0,     0])
agent: i; target: kathy did this intentionally
tensor(0)

tensor(3102)
agent
tensor(10)
:
tensor(3)

tensor(23)
i
tensor(117)
;
tensor(2387)
target
tensor(10)
:
tensor(3)

tensor(1258)
ka
tensor(189)
th
tensor(63)
y
tensor(410)
did
tensor(48)
this
tensor(24522)
intentionally


In [26]:
def find_sub_list(sl, l):
    results = []
    sll = len(sl)
    for ind in (i for i, e in enumerate(l) if e == sl[0]):
        if l[ind:ind+sll] == sl:
            results.append((ind, ind+sll-1))
    return results

def generate_label(input: str, target: str):
    mapper = {
        "O": 0,
        "B-AGENT": 1,
        "I-AGENT": 2,
        "B-DSE": 3,
        "I-DSE": 4,
        "B-TARGET": 5,
        "I-TARGET": 6
    }
    inv_mapper = {v: k for k, v in mapper.items()}

    input = input.split(" ")
    target = target.split("; ")

    init_target_label = [mapper['O']]*len(input)

    for ent in target:
        ent = ent.split(": ")
        try:
            sent_end = ent[1].split(" ")
            index = find_sub_list(sent_end, input)
        except:
            continue
        # print(index)
        try:
            init_target_label[index[0][0]] = mapper[f"B-{ent[0].upper()}"]
            for i in range(index[0][0]+1, index[0][1]+1):
                init_target_label[i] = mapper[f"I-{ent[0].upper()}"]
        except:
            continue
    init_target_label = [inv_mapper[j] for j in init_target_label]
    return init_target_label

In [27]:
from tqdm import tqdm

test_dataset = MPQADataset(tokenizer=tokenizer, dataset=dataset, type_path='test')
test_loader = DataLoader(test_dataset, batch_size=32,
                             num_workers=2, shuffle=True)
model.eval()
model = model.to("cuda")
outputs = []
targets = []
all_text = []
true_labels = []
pred_labels = []
for batch in tqdm(test_loader):
    input_ids = batch['source_ids'].to("cuda")
    attention_mask = batch['source_mask'].to("cuda")
    outs = model.generate(input_ids=input_ids,
                                attention_mask=attention_mask)
    dec = [tokenizer.decode(ids, skip_special_tokens=True,
                            clean_up_tokenization_spaces=False).strip() for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True,  clean_up_tokenization_spaces=False).strip()
                for ids in batch["target_ids"]]
    texts = [tokenizer.decode(ids, skip_special_tokens=True,  clean_up_tokenization_spaces=False).strip()
                for ids in batch["source_ids"]]
    true_label = [generate_label(texts[i].strip(), target[i].strip()) if target[i].strip() != 'none' else [
        "O"]*len(texts[i].strip().split()) for i in range(len(texts))]
    pred_label = [generate_label(texts[i].strip(), dec[i].strip()) if dec[i].strip() != 'none' else [
        "O"]*len(texts[i].strip().split()) for i in range(len(texts))]

    outputs.extend(dec)
    targets.extend(target)
    true_labels.extend(true_label)
    pred_labels.extend(pred_label)
    all_text.extend(texts)

  0%|          | 0/48 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/pai/envs/T5/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 48/48 [00

In [28]:
from datasets import load_metric

metric = load_metric("seqeval")

for i in range(10):
    print(f"Text:  {all_text[i]}")
    print(f"Predicted Token Class:  {pred_labels[i]}")
    print(f"True Token Class:  {true_labels[i]}")
    print("=====================================================================\n")

print(metric.compute(predictions=pred_labels, references=true_labels))

/home/pai/envs/T5/lib/python3.9/site-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Text:  we have been left with no alternative but to buy some weapons via our allies in the drc who have access to some of these european weapons that we have preference for . '' dse:have preference for
Predicted Token Class:  ['B-AGENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
True Token Class:  ['B-AGENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-TARGET', 'I-TARGET', 'I-TARGET', 'I-TARGET', 'I-TARGET', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Text:  all this has sounded the alarm for the us executive and suits the opposition 's objectives to bring about an intervention or a blockade , which would hurt the people it claims to defend . '' dse:claims
Predicted Token Class:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

# Model

Majority of the code here is adapted from [here](https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/t5_fine_tuning.ipynb) which uses the pytorch-lightning framework for training neural networks. T5 has shown that it can generate state of the art on many tasks as long as it can be cast as a text-to-text problem

In [4]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparam):
        super(T5FineTuner, self).__init__()
        self.hparam = hparam

        self.model = T5ForConditionalGeneration.from_pretrained(
            hparam.model_name_or_path)
        self.tokenizer = AutoTokenizer.from_pretrained(
            hparam.model_name_or_path
        )
        self.save_hyperparameters()

    def is_logger(self):
        return True

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            lm_labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparam.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparam.learning_rate, eps=self.hparam.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self,
                       epoch=None,
                       batch_idx=None,
                       optimizer=None,
                       optimizer_idx=None,
                       optimizer_closure=None,
                       on_tpu=None,
                       using_native_amp=None,
                       using_lbfgs=None
                       ):

        optimizer.step(closure=optimizer_closure)
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(
            self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(
            tokenizer=self.tokenizer, type_path="train", args=self.hparam)
        dataloader = DataLoader(train_dataset, batch_size=self.hparam.train_batch_size,
                                drop_last=True, shuffle=True, num_workers=2)
        t_total = (
            (len(dataloader.dataset) //
             (self.hparam.train_batch_size * max(1, self.hparam.n_gpu)))
            // self.hparam.gradient_accumulation_steps
            * float(self.hparam.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparam.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(
            tokenizer=self.tokenizer, type_path="validation", args=self.hparam)
        return DataLoader(val_dataset, batch_size=self.hparam.eval_batch_size, num_workers=2)

In [5]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [6]:
args_dict = dict(
    data_dir="wikiann", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='t5-small',
    tokenizer_name_or_path='t5-small',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=3,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=True, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

# Dataset

Here, I used the popular [WikiANN](https://https://huggingface.co/datasets/wikiann) dataset which is a multilingual named entity recognition dataset consisting of Wikipedia articles annotated with LOC (location), PER (person), and ORG (organisation) tags in the IOB2 format.

In [7]:
from datasets import load_dataset

dataset = load_dataset("wikiann", "en")

In [8]:
print(dataset)

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})


In [9]:
" ".join(dataset['train'][0]['tokens'])

'R.H. Saunders ( St. Lawrence River ) ( 968 MW )'

In [10]:
dataset['train'][0]

{'tokens': ['R.H.',
  'Saunders',
  '(',
  'St.',
  'Lawrence',
  'River',
  ')',
  '(',
  '968',
  'MW',
  ')'],
 'ner_tags': [3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0],
 'langs': ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'],
 'spans': ['ORG: R.H. Saunders', 'ORG: St. Lawrence River']}

In this section, we create a custom dataset class where we cast the NER task as a text to text problem. This is done by concatenating the spans in the data as one line of string separated by a semi-colon (;). e.g

*   **Input**: R.H. Saunders ( St. Lawrence River ) ( 968 MW )
*   **Target**: ORG: R.H. Saunders; ORG: St. Lawrence River




In [11]:
class WikiAnnDataset(Dataset):
  def __init__(self, tokenizer, dataset, type_path, max_len=512):

    self.data = dataset[type_path]
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.tokenizer.max_length = max_len
    self.tokenizer.model_max_length = max_len
    self.inputs = []
    self.targets = []

    self._build()

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

  def _build(self):
    for idx in range(len(self.data)):
      input_, target = " ".join(self.data[idx]["tokens"]), "; ".join(self.data[idx]["spans"])

      input_ = input_.lower() + ' </s>'
      target = target.lower() + " </s>"

       # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [input_], max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
      )
       # tokenize targets
      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target],max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("../T5-base")

print(tokenizer)

input_dataset = WikiAnnDataset(tokenizer=tokenizer, dataset=dataset, type_path='train')

T5TokenizerFast(name_or_path='../T5-base', vocab_size=32100, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42

In [13]:
for i in range(len(input_dataset)):
    _ = input_dataset[i]

In [14]:
data = input_dataset[0]

print(tokenizer.decode(data["source_ids"], skip_special_tokens=False))
print(tokenizer.decode(data["target_ids"], skip_special_tokens=False))

r.h. saunders ( st. lawrence river ) ( 968 mw )</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [15]:
!mkdir -p t5_ner

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
args = argparse.Namespace(**args_dict)
model = T5FineTuner(args)

In [17]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filename=args.output_dir+"/checkpoint.pth", monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    #early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    #amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

# train_params = dict(
#     accumulate_grad_batches=args.gradient_accumulation_steps,
#     ## gpus=args.n_gpu,
#     max_epochs=args.num_train_epochs,
#     #early_stop_callback=False,
#     precision= 16 if args.fp_16 else 32,
#     #amp_level=args.opt_level,
#     gradient_clip_val=args.max_grad_norm,
#     # callbacks=[LoggingCallback()],
# )

In [18]:
def get_dataset(tokenizer, type_path, args):
    tokenizer.max_length = args.max_seq_length
    tokenizer.model_max_length = args.max_seq_length
    dataset = load_dataset(args.data_dir, "en")
    return WikiAnnDataset(tokenizer=tokenizer, dataset=dataset, type_path=type_path)

In [19]:
trainer = pl.Trainer(**train_params)

Using 16bit native Automatic Mixed Precision (AMP)
/home/pai/envs/T5/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f6e53eb2f70>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f6e53eb2f70>)`.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [20]:
trainer.fit(model)

/home/pai/envs/T5/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:102: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every optimization step).
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/pai/envs/T5/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
121.013   To

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/pai/envs/T5/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARAL

/home/pai/envs/T5/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/3750 [00:00<?, ?it/s] 

...

Epoch 2:  67%|██████▋   | 2500/3750 [06:38<03:19,  6.28it/s, loss=0.0849, v_num=0]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/1250 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Epoch 2:  67%|██████▋   | 2502/3750 [06:38<03:18,  6.28it/s, loss=0.0849, v_num=0]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch 2: 100%|██████████| 3750/3750 [07:50<00:00,  7.98it/s, loss=0.0849, v_num=0]


## Load the Stored Model and Evaluate

In [23]:
model = model.load_from_checkpoint("/mnt/workspace/ORL/lightning_logs/version_0/checkpoints/epoch=2-step=470.ckpt")


In [24]:
import textwrap

dataloader = DataLoader(input_dataset, batch_size=32, num_workers=2, shuffle=True)
model.model.eval()
model = model.to("cpu")
outputs = []
targets = []
texts = []
for batch in dataloader:

    outs = model.model.generate(input_ids=batch['source_ids'],
                                attention_mask=batch['source_mask'])
    dec = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip() for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
                for ids in batch["target_ids"]]
    text = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
                for ids in batch["source_ids"]]
    texts.extend(text)
    outputs.extend(dec)
    targets.extend(target)
    break

for i in range(10):
    c = texts[i]
    lines = textwrap.wrap("text:\n%s\n" % c, width=100)
    print("\n".join(lines))
    print("\nActual Entities: %s" % target[i])
    print("Predicted Entities: %s" % outputs[i])
    print("=====================================================================\n")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/pai/envs/T5/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


text: calhoun city , mississippi

Actual Entities: loc: calhoun city , mississippi
Predicted Entities: loc: calhoun city , mississippi

text: honorary fellow of the american institute of architects ( 1993 ) and of the royal institute of
british architects ( 1995 ) .

Actual Entities: org: american institute of architects; org: royal institute of british architects
Predicted Entities: org: american institute of architects; org: royal institute of british architects

text: museo di storia naturale di venezia , venice

Actual Entities: org: museo di storia naturale di venezia; loc: venice
Predicted Entities: loc: museo di storia naturale di venezia

text: edgar allan poe in popular culture

Actual Entities: org: edgar allan poe in popular culture
Predicted Entities: per: edgar allan poe in popular culture

text: '' shine my shoes ''

Actual Entities: org: shine my shoes
Predicted Entities: org: shine my shoes

text: värmlands län , seat no .

Actual Entities: org: värmlands län
Predicted 

In [25]:
def find_sub_list(sl, l):
    results = []
    sll = len(sl)
    for ind in (i for i, e in enumerate(l) if e == sl[0]):
        if l[ind:ind+sll] == sl:
            results.append((ind, ind+sll-1))
    return results

def generate_label(input: str, target: str):
    mapper = {'O': 0, 'B-DATE': 1, 'I-DATE': 2, 'B-PER': 3,
              'I-PER': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-LOC': 7, 'I-LOC': 8}
    inv_mapper = {v: k for k, v in mapper.items()}

    input = input.split(" ")
    target = target.split("; ")

    init_target_label = [mapper['O']]*len(input)

    for ent in target:
        ent = ent.split(": ")
        try:
            sent_end = ent[1].split(" ")
            index = find_sub_list(sent_end, input)
        except:
            continue
        # print(index)
        try:
            init_target_label[index[0][0]] = mapper[f"B-{ent[0].upper()}"]
            for i in range(index[0][0]+1, index[0][1]+1):
                init_target_label[i] = mapper[f"I-{ent[0].upper()}"]
        except:
            continue
    init_target_label = [inv_mapper[j] for j in init_target_label]
    return init_target_label

In [26]:
from tqdm import tqdm

test_dataset = WikiAnnDataset(tokenizer=tokenizer, dataset=dataset, type_path='test')
test_loader = DataLoader(test_dataset, batch_size=32,
                             num_workers=2, shuffle=True)
model.model.eval()
model = model.to("cuda")
outputs = []
targets = []
all_text = []
true_labels = []
pred_labels = []
for batch in tqdm(test_loader):
    input_ids = batch['source_ids'].to("cuda")
    attention_mask = batch['source_mask'].to("cuda")
    outs = model.model.generate(input_ids=input_ids,
                                attention_mask=attention_mask)
    dec = [tokenizer.decode(ids, skip_special_tokens=True,
                            clean_up_tokenization_spaces=False).strip() for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True,  clean_up_tokenization_spaces=False).strip()
                for ids in batch["target_ids"]]
    texts = [tokenizer.decode(ids, skip_special_tokens=True,  clean_up_tokenization_spaces=False).strip()
                for ids in batch["source_ids"]]
    true_label = [generate_label(texts[i].strip(), target[i].strip()) if target[i].strip() != 'none' else [
        "O"]*len(texts[i].strip().split()) for i in range(len(texts))]
    pred_label = [generate_label(texts[i].strip(), dec[i].strip()) if dec[i].strip() != 'none' else [
        "O"]*len(texts[i].strip().split()) for i in range(len(texts))]

    outputs.extend(dec)
    targets.extend(target)
    true_labels.extend(true_label)
    pred_labels.extend(pred_label)
    all_text.extend(texts)

  0%|          | 0/313 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 313/313 [01:18<00:00,  3.97it/s]


In [27]:
all_text[1]

"orguss '' - additional voices"

In [29]:
from datasets import load_metric

metric = load_metric("seqeval")

for i in range(10):
    print(f"Text:  {all_text[i]}")
    print(f"Predicted Token Class:  {pred_labels[i]}")
    print(f"True Token Class:  {true_labels[i]}")
    print("=====================================================================\n")

print(metric.compute(predictions=pred_labels, references=true_labels))

/home/pai/envs/T5/lib/python3.9/site-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Text:  reginald beckwith as kenniston .
Predicted Token Class:  ['B-PER', 'I-PER', 'O', 'B-PER', 'O']
True Token Class:  ['B-PER', 'I-PER', 'O', 'O', 'O']

Text:  orguss '' - additional voices
Predicted Token Class:  ['B-ORG', 'O', 'O', 'O', 'O']
True Token Class:  ['B-ORG', 'O', 'O', 'O', 'O']

Text:  dihedral symmetry groups with even-orders have a number of subgroups .
Predicted Token Class:  ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O']
True Token Class:  ['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Text:  council of southern africa football associations
Predicted Token Class:  ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG']
True Token Class:  ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

Text:  he died in 1924 and was buried in the hôtel des invalides .
Predicted Token Class:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O']
True Token Class:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O